# Opgave 5 — Voorspellen van *Apps* (Deel 2: 5a–5l)

**Doel:** Bouw en valideer een regressiemodel dat het aantal aanmeldingen **Apps** voorspelt op basis van factoren die *vooraf* bekend zijn. Gebruik geen `Accept` of `Enroll`.

**Bronnen (slides & boek):**
- **Lecture 1–5 hand-outs** (kern: L2 distributies & normaliteit, L3 toetsen & splitsen, L4 OLS, L5 diagnostiek, selectie & validatie)
- **Practical Statistics for Data Scientists (PSDS)** – hoofdstukken over regressie, validatie en diagnostiek

> Deze notebook volgt de structuur 5a t/m 5l. Bij elke stap staat een korte verwijzing naar relevante slides/boek.
> **Datum gegenereerd:** 2025-10-09 19:47 (regenerated)


## 5a — Normaliteit van `Apps`
**Bronnen:** Lecture 2 (normality & CLT), Lecture 3 (normality testing), *PSDS* p. 49–52


In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from scipy import stats

df = pd.read_csv("college_statistics.csv")  # Plaats dit bestand naast het notebook
plt.figure(); plt.hist(df["Apps"].dropna(), bins=30)
plt.title("Distribution of Apps"); plt.xlabel("Apps"); plt.ylabel("Frequency"); plt.show()
plt.figure(); stats.probplot(df["Apps"].dropna(), dist="norm", plot=plt)
plt.title("QQ Plot of Apps"); plt.show()
stat, p = stats.shapiro(df["Apps"].dropna())
print(f"Shapiro-Wilk statistic = {stat:.4f}, p-value = {p:.4f}")
df["log_Apps"] = np.log(df["Apps"])
print("Kolommen:", list(df.columns))

## 5b — Estimation/Test-split (600/overig)
**Bron:** Lecture 3 (estimation & test samples)

In [ ]:
from sklearn.model_selection import train_test_split
import random
random.seed(1234)
train, test = train_test_split(df, train_size=600, random_state=1234)
print("Train shape:", train.shape, "Test shape:", test.shape)

## 5c — OLS-modelbouw (log_Apps)
**Bronnen:** Lecture 4 (multiple linear regression), *PSDS* p. 100–107

In [ ]:
import statsmodels.formula.api as smf
base_formula = "log_Apps ~ SAT + Top10perc + Room.Board + Expend + Grad.Rate"
model_base = smf.ols(base_formula, data=train).fit()
print(model_base.summary())

## 5d — RMSE op log-schaal
**Bronnen:** Lecture 5 (model fit), *PSDS* p. 121

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
y_pred_log = model_base.predict(test)
rmse_log = float(np.sqrt(mean_squared_error(test["log_Apps"], y_pred_log)))
print(f"Test RMSE (log-scale) = {rmse_log:.3f}")

## 5e — Diagnostiek (residuen, BP, DW, VIF)
**Bronnen:** Lecture 5 (diagnostics), *PSDS* p. 122–130

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import patsy

resid = model_base.resid
plt.figure(); sm.qqplot(resid, line="45"); plt.title("QQ Plot of Residuals (Base Model)"); plt.show()

bp = sm.stats.diagnostic.het_breuschpagan(resid, model_base.model.exog)
print(f"Breusch–Pagan p-value = {float(bp[1]):.4f}")
dw = sm.stats.stattools.durbin_watson(resid)
print(f"Durbin–Watson = {dw:.2f}")

y_mat, X_mat = patsy.dmatrices(base_formula, train, return_type="dataframe")
vif_table = pd.DataFrame({
    "variable": X_mat.columns,
    "VIF": [variance_inflation_factor(X_mat.values, i) for i in range(X_mat.shape[1])]
}).sort_values("VIF", ascending=False)
print(vif_table)

## 5f — Modelselectie (AIC, all-subsets)
**Bronnen:** Lecture 5 (AIC), *PSDS* p. 134–136

In [ ]:
import itertools
candidate_vars = ["SAT", "Top10perc", "Room.Board", "Expend", "Grad.Rate"]
def best_aic_subset(y, Xvars, data):
    best = {"aic": float("inf"), "model": None, "vars": None, "formula": None}
    for k in range(1, len(Xvars)+1):
        for combo in itertools.combinations(Xvars, k):
            f = f"{y} ~ " + " + ".join(combo)
            m = smf.ols(f, data=data).fit()
            if m.aic < best["aic"]:
                best = {"aic": float(m.aic), "model": m, "vars": combo, "formula": f}
    return best
best_sub = best_aic_subset("log_Apps", candidate_vars, train)
print("Best subset:", best_sub["vars"], "| AIC:", round(best_sub["aic"], 2))
print(best_sub["model"].summary())

## 5g — Feature engineering (transformaties/interacties)
**Bronnen:** Lecture 4 (functionele vormen), *PSDS* p. 111–113

In [ ]:
train["log_Expend"] = np.log(train["Expend"]); test["log_Expend"] = np.log(test["Expend"])
fe_formula = "log_Apps ~ SAT + I(SAT**2) + Top10perc + Room.Board + log_Expend + Grad.Rate + SAT:Top10perc"
model_fe = smf.ols(fe_formula, data=train).fit()
print("FE AIC:", round(model_fe.aic, 2)); print(model_fe.summary())

## 5h — K-fold cross-validation (5-fold RMSE)
**Bronnen:** Lecture 5 (validation techniques), *PSDS* p. 139

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def cv_rmse(formula, data, k=5, random_state=1234):
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    rmses = []
    for tr_idx, va_idx in kf.split(data):
        tr = data.iloc[tr_idx]; va = data.iloc[va_idx]
        m = smf.ols(formula, data=tr).fit()
        pred = m.predict(va)
        rmses.append(float(np.sqrt(mean_squared_error(va["log_Apps"], pred))))
    return float(np.mean(rmses)), float(np.std(rmses))

sub_formula = best_sub["formula"]
cv_sub = cv_rmse(sub_formula, train)
cv_fe  = cv_rmse(fe_formula,   train)
print("CV RMSE (subset):", cv_sub)
print("CV RMSE (FE):    ", cv_fe)

## 5i — Robuuste standaardfouten (HC3)
**Bronnen:** Lecture 5 (robust inference), *PSDS* p. 125

In [ ]:
prefer_formula = fe_formula if cv_fe[0] <= cv_sub[0] else sub_formula
model_pref = smf.ols(prefer_formula, data=train).fit()
model_pref_HC3 = model_pref.get_robustcov_results(cov_type="HC3")
print("Prefered formula:", prefer_formula)
print("AIC (non-robust):", round(model_pref.aic, 2))
print(model_pref_HC3.summary())

## 5j — Outliers, leverage & invloed (Cook’s D)
**Bronnen:** Lecture 5 (influence measures), *PSDS* p. 128–130

In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence
infl = OLSInfluence(model_pref)
cooks_d = infl.cooks_distance[0]; lev = infl.hat_matrix_diag
n = len(train); p = model_pref.df_model + 1; thr = 4 / (n - p)
idx = np.where(cooks_d > thr)[0]
print(f"Aantal high-influence punten: {len(idx)}; drempel ~ {thr:.5f}")
if len(idx) > 0:
    train_sens = train.drop(train.index[idx])
    model_sens = smf.ols(prefer_formula, data=train_sens).fit()
    import pandas as pd
    comp = pd.DataFrame({"orig": model_pref.params, "sens": model_sens.params})
    print(comp.head(8))

## 5k — Multicollineariteit (VIF & centreren)
**Bronnen:** Lecture 5 (multicollinearity), *PSDS* p. 120

In [ ]:
import pandas as pd
y0, X0 = patsy.dmatrices(prefer_formula, train, return_type="dataframe")
vif0 = pd.DataFrame({"variable": X0.columns,
                     "VIF": [variance_inflation_factor(X0.values, i) for i in range(X0.shape[1])]}
                   ).sort_values("VIF", ascending=False)
print("VIF voor centreren:"); print(vif0.head(10))

cols_to_center = []
for c in ["SAT", "Top10perc", "Room.Board", "log_Expend", "Grad.Rate"]:
    if c in train.columns:
        cols_to_center.append(c)
        train[c + "_c"] = train[c] - train[c].mean()
        test[c + "_c"]  = test[c]  - train[c].mean()

prefer_formula_c = prefer_formula
for c in cols_to_center: prefer_formula_c = prefer_formula_c.replace(c, c + "_c")

import statsmodels.formula.api as smf
model_pref_c = smf.ols(prefer_formula_c, data=train).fit()

y1, X1 = patsy.dmatrices(prefer_formula_c, train, return_type="dataframe")
vif1 = pd.DataFrame({"variable": X1.columns,
                     "VIF": [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]}
                   ).sort_values("VIF", ascending=False)
print("VIF na centreren:"); print(vif1.head(10))
print(model_pref_c.summary())

## 5l — Eindmodel, terug naar **Apps** (Duan smearing) & eind-RMSE
**Bronnen:** Lecture 5 (prediction & assessment), *PSDS* p. 140

In [ ]:
from sklearn.metrics import mean_squared_error
final_model = model_pref_c  # kies je eindmodel hier expliciet

train_pred_log = final_model.predict(train)
smearing = float(np.mean(np.exp(train["log_Apps"] - train_pred_log)))

test_pred_log = final_model.predict(test)
test_pred_apps = np.exp(test_pred_log) * smearing

rmse_apps = float(np.sqrt(mean_squared_error(test["Apps"], test_pred_apps)))
print(f"Final test RMSE (Apps-scale) = {rmse_apps:,.2f}")

baseline_rmse = float(np.sqrt(mean_squared_error(test["Apps"], np.repeat(train['Apps'].mean(), len(test)))))
print(f"Baseline RMSE (predict mean) = {baseline_rmse:,.2f}")
print(f"Verbetering t.o.v. baseline: {100*(1 - rmse_apps/baseline_rmse):.1f}%")

def predict_apps(new_df, train_ref=train, model=final_model, smear=smearing):
    import numpy as np, pandas as pd
    new_df = new_df.copy()
    if "Expend" in new_df and "log_Expend" not in new_df:
        new_df["log_Expend"] = np.log(new_df["Expend"])
    for c in ["SAT", "Top10perc", "Room.Board", "log_Expend", "Grad.Rate"]:
        if c in new_df and (c + "_c") in train_ref.columns:
            new_df[c + "_c"] = new_df[c] - train_ref[c].mean()
    log_hat = model.predict(new_df)
    return np.exp(log_hat) * smear

# Voorbeeld:
# new = pd.DataFrame({"SAT":[1200], "Top10perc":[20], "Room.Board":[10000], "Expend":[8000], "Grad.Rate":[70]})
# print(predict_apps(new))